In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [10]:
startDay = '20191206'

datetimeStartDay = datetime.strptime(startDay, '%Y%m%d')

small_df = []
for i in range(0,823):
    nextDay = datetimeStartDay + timedelta(days=i)
    onlyDay = nextDay.strftime('%Y%m%d')
    response = requests.get('https://api.weather.com/v1/location/EKYT:9:DK/observations/historical.json?apiKey=e1f10a1e78da46f5b10a1e78da96f525&units=m&startDate='+onlyDay)
    jsonData = response.json()
    df = pd.json_normalize(jsonData['observations'])
    df = df[['valid_time_gmt', 'heat_index', 'rh', 'pressure', 'wdir', 'wdir_cardinal', 'wspd']]
    small_df.append(df)
    response.close()

concatdf = pd.concat(small_df, ignore_index=True)
#concatdf.to_csv('Dataset/WeatherData19des-22mar.csv', index=False, sep=',')

In [34]:

#concatdf = pd.read_csv('Dataset/WeatherData19des-22mar.csv')


for i in range(0, concatdf.shape[0]):
    timeChange = concatdf.loc[i,'valid_time_gmt']
    convertTime = datetime.utcfromtimestamp(timeChange)
    convertTime = convertTime + timedelta(hours=2)
    concatdf.loc[i, 'valid_time_gmt'] = convertTime
#concatdf.to_csv('Dataset/NOTRemoveBadTimes.csv', index=False)

In [49]:
#concatdf = pd.read_csv('Dataset/NOTRemoveBadTimes.csv')
concatdf['valid_time_gmt'] = pd.to_datetime(concatdf['valid_time_gmt'])

ts = pd.date_range("2019-12-06 00:00:00", "2022-03-09 00:00:00", freq='10min')
print(concatdf.columns)
concatdf =concatdf.set_index('valid_time_gmt').reindex(ts).fillna(method="ffill").rename_axis('valid_time_gmt').reset_index()

#df[(df.groupby(["valid_time_gmt"], as_index=False)["valid_time_gmt"].diff().fillna(pd.Timedelta(seconds=0)).dt.seconds <= 1000).reset_index(drop=True)]
print(concatdf.shape)
#concatdf.to_csv("teswtaaaaing.csv",index=False)



Index(['valid_time_gmt', 'heat_index', 'rh', 'pressure', 'wdir',
       'wdir_cardinal', 'wspd'],
      dtype='object')
(118657, 7)


In [51]:
#concatdf = pd.read_csv("teswtaaaaing.csv")

ts = pd.date_range("2019-12-06 00:00:00", "2022-03-09 00:00:00", freq='30min')
concatdf['valid_time_gmt'] = pd.to_datetime(concatdf['valid_time_gmt'])
concatdf.index = concatdf["valid_time_gmt"]
df2 = pd.DataFrame(index=ts)
print(df2.index)
df3 =df2.join(concatdf)
#df3 = pd.concat([df,df2],axis=1)
df3.to_csv("Dataset/Weather_Data.csv",index=False)

DatetimeIndex(['2019-12-06 00:00:00', '2019-12-06 00:30:00',
               '2019-12-06 01:00:00', '2019-12-06 01:30:00',
               '2019-12-06 02:00:00', '2019-12-06 02:30:00',
               '2019-12-06 03:00:00', '2019-12-06 03:30:00',
               '2019-12-06 04:00:00', '2019-12-06 04:30:00',
               ...
               '2022-03-08 19:30:00', '2022-03-08 20:00:00',
               '2022-03-08 20:30:00', '2022-03-08 21:00:00',
               '2022-03-08 21:30:00', '2022-03-08 22:00:00',
               '2022-03-08 22:30:00', '2022-03-08 23:00:00',
               '2022-03-08 23:30:00', '2022-03-09 00:00:00'],
              dtype='datetime64[ns]', length=39553, freq='30T')
